In [2]:
import random
import re

import pandas as pd
import torch

from collections import defaultdict

from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from tqdm import trange, tqdm

/home/konstfed/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ENTITIES = ['PERSON',
 'PROFESSION',
 'ORGANIZATION',
 'EVENT',
 'DATE',
 'COUNTRY',
 'CITY',
 'NUMBER',
 'AGE',
 'ORDINAL',
 'NATIONALITY',
 'FACILITY',
 'STATE_OR_PROVINCE',
 'LAW',
 'AWARD',
 'LOCATION',
 'IDEOLOGY',
 'WORK_OF_ART',
 'PRODUCT',
 'CRIME',
 'DISEASE',
 'TIME',
 'MONEY',
 'DISTRICT',
 'PENALTY',
 'RELIGION',
 'PERCENT',
 'LANGUAGE',
 'FAMILY']

In [4]:
train_json = pd.read_json("../data/public_dat/train.jsonl", lines=True)
train_json

,ners,sentences,id
0,"[[0, 5, CITY], [16, 23, PERSON], [34, 41, PERS...",Бостон взорвали Тамерлан и Джохар Царнаевы из ...,0
1,"[[21, 28, PROFESSION], [53, 67, ORGANIZATION],...",Умер избитый до комы гитарист и сооснователь г...,1
2,"[[0, 4, PERSON], [37, 42, COUNTRY], [47, 76, O...",Путин подписал распоряжение о выходе России из...,2
3,"[[0, 11, PERSON], [36, 47, PROFESSION], [49, 6...",Бенедикт XVI носил кардиостимулятор\nПапа Римс...,3
4,"[[0, 4, PERSON], [17, 29, ORGANIZATION], [48, ...",Обама назначит в Верховный суд латиноамериканк...,4
...,...,...,...
514,"[[42, 46, COUNTRY], [82, 87, COUNTRY], [104, 1...",Глава Малайзии: мы не хотим противостоять Кита...,514
515,"[[1, 4, PRODUCT], [31, 33, FACILITY], [35, 44,...",«Союз» впервые пристыковался к МКС за 6 часов\...,515
516,"[[0, 4, PERSON], [8, 12, PERSON], [45, 52, AGE...",Трамп и Путин сделали совместное заявление к 7...,516
517,"[[0, 9, NATIONALITY], [58, 72, PERSON], [101, ...",Российский магнат устроил самую дорогую свадьб...,517


In [5]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
print(tokenizer.encode("что то Тут написал [PAD]", add_special_tokens=True, return_tensors='pt'))
print(tokenizer.decode([101, 1997, 3815, 29955, 12715,0, 102]))
print(tokenizer.decode([1997, 3815, 29955, 12715]))

tensor([[  101,  1997,  3815, 29955, 12715,     0,   102]])
[CLS] что то Тут написал [PAD] [SEP]
что то Тут написал


In [6]:
def _split_long_seq(
    input_ids,
    labels,
    max_length: int,
    cls_id: int = 101,
    sep_id: int = 102,
    pad_id: int = 0,
) -> tuple[torch.Tensor, list[str]]:
    labels = labels[1:-1]
    input_ids = input_ids[:, 1:-1]  # remove special tokens from beginning and end
    new_input_ids = []
    new_labels = []
    for i in range(0, input_ids.shape[1], max_length - 2):
        ids = input_ids[:, i : min(i + max_length - 2, input_ids.shape[1])]
        c_ids = torch.zeros((ids.shape[0], max_length), dtype=torch.long).type(torch.LongTensor)
        c_ids[:, 0] = cls_id
        c_ids[:, 1:1 + ids.shape[1]]
        c_ids[:, 1 + ids.shape[1]] = sep_id
        c_ids[:, 2 + ids.shape[1]: ] = pad_id
 
        c_labels = (
            ["[CLS]"]
            + labels[i : min(i + max_length - 2, input_ids.shape[0])]
            + ["[SEP]"]
        )
        new_input_ids.append(c_ids)
        new_labels.append(c_labels)

    return new_input_ids, new_labels

In [30]:

from torch import nn
import torch.utils
import torch.utils.data


def _get_unique_entities_seq(entities: list[str], labels: list[str]) -> dict:
    was = set()
    for l in labels:
        if l in entities:
            was.add(l)
    
    new_labels = {}
    for ent in was:
        c_seq = []
        for i in range(len(labels)):
            if labels[i] == ent:
                c_seq.append(1)
            else:
                c_seq.append(0)
        new_labels[ent] = c_seq
    return new_labels

class NestedNERDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, entities: list[str]):
        self.df = df
        self.entities = entities
        self.tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
        self.bert_model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")
        self.tokenized_ds = []
        self.tokenized_per_entity = []
        self._tokenize_per_class()
        self._get_word_embds()

    def _tokenize_per_class(self):
        # for ent in self.entities:
        #     self.tokenized_per_entity.append([])

        for _, data in tqdm(self.df.iterrows(), total=self.df.shape[0]):
            sentence = data["sentences"]
            ners = data["ners"]
            tokenized_sentence = self.tokenizer.tokenize(sentence)
            # print("\n")
            # print(tokenized_sentence)
            tokenized_length = len(tokenized_sentence)

            tokens = []
            labels = []

            for _ in range(tokenized_length):
                labels.append("O")  # "O" represents tokens outside entities

            for start, end, label in ners:
                # Convert character-level annotations to token-level
                start_token = self.tokenizer.encode(
                    sentence[:start], add_special_tokens=False
                )
                end_token = self.tokenizer.encode(
                    sentence[: end + 1], add_special_tokens=False
                )
                # print(sentence[start:end+1], ":", len(start_token), len(end_token), "-->", len(labels))
                # Update labels for tokens within the entity span
                for i in range(len(start_token), len(end_token)):
                    labels[i] = label

            labels = ["[CLS]"] + labels + ["[SEP]"]
            input_ids = self.tokenizer.encode(
                tokenized_sentence,
                add_special_tokens=True,
                return_tensors="pt",
                padding="max_length",
                max_length=512,
            )
            if input_ids.shape[1] > 512:
                # print("WAS", input_ids.shape)
                input_ids, labels = _split_long_seq(input_ids, labels, 512)
                for i in range(len(labels)):
                    # print(input_ids[i].shape)
                    self.tokenized_ds.append(
                        {
                            "input_ids": input_ids[i],
                            "attention_mask": torch.ones_like(input_ids[i]),
                            "labels": labels[i]
                        }
                    )
            else:
                attention_mask = torch.ones_like(input_ids)
                self.tokenized_ds.append(
                    {
                        "input_ids": input_ids,
                        "attention_mask": attention_mask,
                        "labels": labels,
                    }
                )

    def _get_word_embds(self, batch_size: int = 4):
        # all_tokens = pass
        all_tokens = [x["input_ids"] for x in self.tokenized_ds]
        attention_masks = [x["attention_mask"] for x in self.tokenized_ds]
        all_labels = [x["labels"] for x in self.tokenized_ds]
        
        ds = {x:([], []) for x in self.entities}
        
        with torch.no_grad():
            for i in trange(0, len(all_tokens), batch_size):
                # print([x.shape for x in all_tokens[i : min(i + batch_size, len(all_tokens))]])
                end_idx = min(i + batch_size, len(all_tokens))
                input_ids = torch.cat(
                    all_tokens[i :end_idx]
                )
                att_mask = torch.cat(
                    attention_masks[i : end_idx]
                )
                labels = all_labels[i : end_idx]
                outputs = self.bert_model(input_ids, attention_mask=att_mask)
                word_embeddings = outputs.last_hidden_state
                
                word_embeddings = word_embeddings[:, 1:-1]
    
                for j in range(len(labels)):
                    seqs = _get_unique_entities_seq(self.entities, labels[j][1:-1])
                    for ent_name in seqs:
                        # sequence labels
                        for k in range(len(seqs[ent_name])):
                            # ds[ent_name][1].append(seqs[ent_name][k])
                            if seqs[ent_name][k] == 0:
                                ds[ent_name][0].append(word_embeddings[j][k])
                            else:
                                ds[ent_name][1].append(word_embeddings[j][k])

        self.per_entity = {}
        for ent in ds:
            negative_words, positive_samples = ds[ent]
            if len(negative_words) == 0 or len(positive_samples) == 0:
                print(ent, "does have problems")
                continue
            
            negative_words = torch.stack(negative_words)
            positive_samples = torch.stack(positive_samples)
            negative_words = negative_words[torch.randperm(negative_words.shape[0])]
            
            negative_words = negative_words[:positive_samples.shape[0]]
            
            c_labels = torch.cat([torch.ones(positive_samples.shape[0]), torch.zeros(positive_samples.shape[0])])
            c_word_embds = torch.cat([positive_samples, negative_words])
            self.per_entity[ent] = torch.utils.data.TensorDataset(c_word_embds, c_labels)

In [31]:
ds = NestedNERDataset(train_json, ENTITIES)
# ds.tokenized_ds[0]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 135/135 [05:00<00:00,  2.23s/it]


In [33]:
class SimpleModel(nn.Module):
    def __init__(self, emb_dim: int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.cls = nn.Sequential(
            nn.Linear(emb_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.cls.forward(x)

In [40]:
from sklearn.metrics import classification_report

def train_one_epoch(model: nn.Module, optimizer: torch.optim.Optimizer, loss, train_loader):
    model.train()
    avg_loss = 0
    cnt = 0
    bar = tqdm(train_loader)
    for x, label in bar:
        optimizer.zero_grad()
        out = model(x)
        l = loss(out.flatten(), label.flatten())
        l.backward()
        avg_loss = (avg_loss * cnt + l.cpu().item()) / (cnt + 1)
        cnt += 1
        bar.set_postfix({"avg. loss": avg_loss})
        optimizer.step()
        
def test_epoch(model: nn.Module, loss, loader):
    model.eval()
    avg_loss = 0
    cnt = 0
    bar = tqdm(loader)
    gt = []
    pred = []
    for x, label in bar:
        out = model(x)
        l = loss(out.flatten(), label.flatten())
        avg_loss = (avg_loss * cnt + l.cpu().item()) / (cnt + 1)
        cnt += 1
        bar.set_postfix({"avg. loss": avg_loss})
        
        c_pred = out > 0.5
        pred = pred + c_pred.cpu().detach().tolist()
        gt = gt + label.cpu().detach().tolist()
    
    print(classification_report(gt, pred))

def train(model: nn.Module, train_loader, test_loader, n_iter: int = 10):
    optimizer = torch.optim.Adam(model.parameters())
    loss = nn.BCELoss()
    for i in range(n_iter):
        print(f"Epoch {i}:")
        train_one_epoch(model, optimizer, loss, train_loader)
    print("On test:")
    test_epoch(model, loss, test_loader)
    

In [43]:
models = {}

for ent in ds.per_entity:
    print(f"Train {ent}")
    c_m = SimpleModel(768)
    train_ds, test_ds = torch.utils.data.random_split(ds.per_entity[ent], [0.8, 0.2])
    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=20)
    test_loader = torch.utils.data.DataLoader(train_ds, batch_size=20)

    train(c_m, train_loader, test_loader, 3)
    models[ent] = c_m

Train PERSON
Epoch 0:


100%|██████████| 971/971 [00:02<00:00, 438.44it/s, avg. loss=0.0694]


Epoch 1:


100%|██████████| 971/971 [00:02<00:00, 480.69it/s, avg. loss=0.0344]


Epoch 2:


...

Epoch 2:


100%|██████████| 5/5 [00:00<00:00, 233.51it/s, avg. loss=0.28]


On test:


100%|██████████| 5/5 [00:00<00:00, 523.91it/s, avg. loss=0.213]


              precision    recall  f1-score   support

         0.0       0.98      0.95      0.97        44
         1.0       0.95      0.98      0.97        43

    accuracy                           0.97        87
   macro avg       0.97      0.97      0.97        87
weighted avg       0.97      0.97      0.97        87



In [87]:
train_json.iloc[42].sentences

'Теракт в Иерусалиме: убиты двое полицейских\nПогибшие в результате теракта полицейские\nВ пятницу, 14 июля 2017 года, в Восточном Иерусалиме рядом с Храмовой горой произошёл теракт.\n\nУтром трое неизвестных неожиданно начали стрелять возле Львиных ворот Старого города, после чего попытались убежать и скрыться в одной из мечетей на Храмовой горе.\n\nРаненых полицейских и пограничника госпитализировали в больницу Адаса Хар-ха-Цофим (בית החולים הדסה הר הצופים). Однако двое из них в полдень умерли.\n\nПосле нападения вход на Храмовую гору был закрыт, а все посетители\xa0— эвакуированы.\n\nПосле 11:30 появились сообщения о том, что вооруженное нападение на Храмовой горе совершили трое жителей израильского арабского города Умм эль-Фахм.\n\nВсе трое, по документам, носили одно имя\xa0— Мухаммад Джабарин. Им было 30, 20 и 19 лет. Полные имена террористов: Мухаммад Ахмад Мухаммад Джабарин, Мухаммад Хамад Абд аль-Латиф Джабарин и Мухаммад Ахмад Мафаль Джабарин.\n\nДвое из преступников за день 

In [108]:
class Pipeline:
    def __init__(self, bert_model, bert_tokenizer, models) -> None:
        self.bert_model: BertModel = bert_model
        self.bert_tokenizer: BertTokenizer = bert_tokenizer
        self.models = models

    def forward(self, x: str):
        tokenized = self.bert_tokenizer.batch_encode_plus(
            [x], add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512
        )
        # if tokenized["input_ids"]
        if tokenized["input_ids"].shape[1] > 512:
            print("POMOGITE")
        output = self.bert_model(
            tokenized["input_ids"], attention_mask=tokenized["attention_mask"]
        )
        word_embeddings = output.last_hidden_state[0]

        tokens = tokenizer.convert_ids_to_tokens(tokenized["input_ids"][0].tolist())
        tokens_lengths = list(map(lambda x: len(x.replace("#", "")), tokens))
        tokens_lengths[0] = 0
        tokens_lengths[-1] = 0

        nested_ners = []

        for ent in self.models:
            labels = self.models[ent](word_embeddings)
            labels = labels[1:-1]  # remove [CLS] and [SEP]
            c_seq = labels > 0.5
            i = 0
            # print(ent)
            # print(labels)
            # print(c_seq)
            while i < len(c_seq):
                if c_seq[i] == 0:
                    i += 1
                    continue

                j = i + 1
                while j < len(c_seq):
                    if c_seq[j] != c_seq[i]:
                        break

                    j += 1

                if i + 1 == j:
                    i += 1
                    continue

                nested_ners.append((i, j - 1, ent))
                i = j

        return nested_ners


pipe = Pipeline(ds.bert_model, ds.tokenizer, models)
# to_test = train_json.iloc[42]
# to_test = submission_json.iloc[2].senences
# result = pipe.forward(to_test)

# for l_idx, r_idx, ent in result:
#     print(f"{ent}: ({l_idx}, {r_idx}) --> {to_test[l_idx: r_idx + 1]}")

In [141]:
submission_json = pd.read_json("../data/test/test.jsonl", lines=True)
submission_json.head()

,senences,id
0,Владелец «Бирмингема» получил шесть лет тюрьмы...,584
1,Акция протеста на Майдане Независимости объявл...,585
2,Фольксваген может перейти под контроль Порше \...,586
3,В Москве покажут фильмы Чарли Чаплина с живой ...,587
4,Чулпан Хаматова сыграет главную роль в фильме ...,588


In [142]:
submission_json.shape

(65, 2)

In [143]:
submission_results = []
for i, (sent, id) in tqdm(submission_json.iterrows(), total=submission_json.shape[0]):
    results = pipe.forward(sent)
    submission_results.append({"id": id, "ners": results})
    

100%|██████████| 65/65 [00:24<00:00,  2.65it/s]


In [144]:
import json

with open("test.jsonl", "w") as f:
    for i in range(len(submission_results)):
        json.dump(submission_results[i], f)
        f.write("\n")
    

In [138]:
subm = pd.read_json("test.jsonl", lines=True)
subm

,id,ners
0,519,"[[1, 4, PERSON], [12, 14, PERSON], [39, 42, PE..."
1,520,"[[18, 21, PERSON], [36, 37, PERSON], [45, 48, ..."
2,521,"[[1, 2, PERSON], [9, 12, PERSON], [16, 17, PER..."
3,522,"[[26, 28, PERSON], [178, 180, PERSON], [266, 2..."
4,523,"[[4, 5, PERSON], [13, 14, PERSON], [21, 22, PE..."
...,...,...
60,579,"[[10, 12, PERSON], [30, 31, PERSON], [33, 34, ..."
61,580,"[[2, 3, PERSON], [12, 13, PERSON], [19, 21, PE..."
62,581,"[[9, 11, PERSON], [13, 14, PERSON], [16, 17, P..."
63,582,"[[4, 9, PERSON], [37, 39, PERSON], [47, 49, PE..."


In [139]:
test_json = pd.read_json("/home/konstfed/Documents/Study/NLP_course/NLP_assignments/assignment3/data/test/test.jsonl", lines=True)
test_json

,senences,id
0,Владелец «Бирмингема» получил шесть лет тюрьмы...,584
1,Акция протеста на Майдане Независимости объявл...,585
2,Фольксваген может перейти под контроль Порше \...,586
3,В Москве покажут фильмы Чарли Чаплина с живой ...,587
4,Чулпан Хаматова сыграет главную роль в фильме ...,588
...,...,...
60,ОБСЕ назвала референдум о статусе Крыма незако...,644
61,Египетского студента могут выслать из страны з...,645
62,Геннадий Онищенко отправлен в отставку\nГеннад...,646
63,Племянник Алишера Усманова разбился в ДТП\nВид...,647
